# Analyzing monochromator QE sweeps.
Craig Lage 28-Aug-24

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import json 
from astropy.time import Time, TimeDelta
from lsst.geom import Point2D
from lsst.daf.butler import Butler
from lsst_efd_client import EfdClient
from lsst.summit.utils import BestEffortIsr
from lsst.resources import ResourcePath
from lsst.summit.utils import quickSmooth
%matplotlib inline

In [ ]:
butler = Butler('/repo/embargo', collections=["LATISS/raw/all", "LATISS/calib", "LATISS/calib/unbounded"])
best_effort_isr = BestEffortIsr(embargo=True)
best_effort_isr.doWrite = False  # Don't write to butler database

client = EfdClient('usdf_efd')

# Get the lfa summary files

In [ ]:
start = Time("2024-08-27T18:00:00", scale='utc')
end = Time("2024-08-28T04:00:00", scale='utc')

lfas = await client.select_time_series('lsst.sal.Script.logevent_largeFileObjectAvailable', \
                                    ['id', 'url'],  start, end)
filenames = []
for i in range(len(lfas)):
    if lfas.iloc[i]['id'].split('_')[0] == 'BL294': # This block is for filter sweeps
        url = lfas.iloc[i]['url']
        filename = url.replace('https://s3.cp.lsst.org/', 's3://rubin:') # Change from summit to USDF
        filenames.append(filename)
filenames    

# Now process the measured images

In [ ]:
fig = plt.figure(figsize=(8,8))
for filename in filenames:
    rp = ResourcePath(filename)
    with rp.open(mode='r') as f:
        data = json.load(f)
    waves = []
    ratios = []
    fig.add_subplot(1,1,1)
    for step in data["steps"]:
        wave = step['wavelength']
        waves.append(wave)
        expIdFilter = int(list(step["latiss_exposure_info"].keys())[0])
        dataId = {'exposure': expIdFilter, 'detector': 0, "instrument": 'LATISS'}
        exp = best_effort_isr.getExposure(dataId, skipCosmics=False)
        data = quickSmooth(exp.image.array, 1)
        medFilter = np.median(data)
        expIdNoFilter = int(list(step["latiss_exposure_info"].keys())[1])
        dataId = {'exposure': expIdNoFilter, 'detector': 0, "instrument": 'LATISS'}
        exp = best_effort_isr.getExposure(dataId, skipCosmics=False)
        data = quickSmooth(exp.image.array, 1)
        medNoFilter = np.median(data)
        ratio = medFilter / medNoFilter
        ratios.append(ratio)
        print(wave, ratio)
    min_wave = np.min(waves)
    max_wave = np.max(waves)
    if min_wave < 400:
        band = 'g'
        color='green'
    else:
        band = 'r'
        color='red'
    # Get the tabulated transmission
    transdata = butler.get("transmission_filter", dataId={'instrument': 'LATISS', 'physical_filter': f'SDSS{band}_65mm~empty'})
    tab_waves = np.linspace(min_wave, max_wave, 500)
    tab_trans = transdata.sampleAt(Point2D(0), tab_waves * 10.0) # Data is in Angstroms
    plt.title(f"AuxTel SDSS{band}_65mm Transmission curve - 28-Aug-24")
    plt.plot(waves, np.array(ratios) * 100.0, marker='x', color=color, label="Measured")
    plt.plot(tab_waves, tab_trans * 100.0, color='black', ls='--', label="Tabulated")
    plt.xlabel("Wavelength (nm)")
    plt.ylabel("Transmittance (%)")
    plt.xlim(min_wave, max_wave)
    plt.ylim(0,100.0)
    plt.legend()
    plt.savefig(f"/home/c/cslage/u/AuxTel/filter_data/SDSS{band}_65mm_Filter_Curve_28Aug24.png")
    plt.clf()